<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [ ]:
# TODO - Words, words, mere words, no matter from the heart.

In [183]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

import random
import sys

import numpy as np

In [234]:
import requests

url = "https://www.gutenberg.org/files/100/100-0.txt"

r = requests.request('POST', url)

r.encoding = r.apparent_encoding

In [245]:
texts = r.text.replace('\r','').split('\n')

t = []
for text in texts:
    if text != '':
        t.append(text)
        
print(len(t))
t0 = t[63:(len(t)-285)]

data = ""
for text in t0:
    data += text

139330


In [247]:
len(t0)

138982

In [248]:
chars = list(set(data))
char_int = {c:i for i,c in enumerate(chars)}
int_char = {i:c for i,c in enumerate(chars)}

maxlen = 40
step = 5

encoded = [char_int[c] for c in data]

sequences = [] # Each element is 40 characters long
next_chars = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_chars.append(encoded[i + maxlen])
print('sequences:', len(sequences))

sequences: 1077121


In [249]:
lim = 500000 # limiter to decrease work size

x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)[:lim]
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)[:lim]

for i, sequence in enumerate(sequences[:lim]):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_chars[i]] = 1

In [250]:
x.shape

(500000, 40, 101)

In [251]:
y.shape

(500000, 101)

In [269]:
model_iter = 0

In [253]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [267]:
model_num = 0

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    # Save a checkpoint of this models epoch
    checkpoint_path = "./checkpoints/model{}_epoch{}.ckpt".format(model_num,epoch)
    model.save_weights(checkpoint_path)
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_int[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = int_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [268]:
load_model = False

if load_model:
    try:
        model = tf.keras.models.load_model('model{}.h5'.format(model_num))
    except Exception as e:
        print(e)
        model.fit(x, y,
                  batch_size=128,
                  epochs=10,
                  callbacks=[print_callback])
else:
    model.fit(x, y,
              batch_size=128,
              epochs=10,
              callbacks=[print_callback])

Epoch 1/10
499968/500000 [============================>.] - ETA: 0s - loss: 1.5618
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "ns that no one owns a United States copy"
ns that no one owns a United States copy    The rease the comes the great of the common the comes    The forth the crown of the street of the street    The stort of the stort of the stand and the comes    The street of the the street of the parter,    And the common the rease the come and the then the great of the field.                                                                                                                      
----- diversity: 0.5
----- Generating with seed: "ns that no one owns a United States copy"
ns that no one owns a United States copy    The great in the reppority    That content the thought the with the world.                                                           Exeunt FOLSTAFFORD  ELIA. 'Twas so for their compleach and the free sir,    Th

c:\users\dylan nason\anaconda3\envs\u4-s2-nn\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


ot.CITIZENSQe.Wed the Gaustimened feellippies-filst. sirch, cud,Sitors! For thy offereother. I was us;The three peacebloys behopl! what natthe art.Wh’le ponjected you stad; but notly to call,Thange isatious Henrishs is?ANGIAN.It, Sufferumphe hast greats; are st, boindly;Heave I candends he was. 
500000/500000 [==============================] - 1007s 2ms/sample - loss: 1.4703
Epoch 10/10
499968/500000 [============================>.] - ETA: 0s - loss: 1.4630
----- Generating text after Epoch: 9
----- diversity: 0.2
----- Generating with seed: "ns that no one owns a United States copy"
ns that no one owns a United States copy    That shall be son the courts of the bellow.                                                                                                                                                                                                                                                                                                                                  

In [265]:
import h5py

In [ ]:
model.save('model{}_iter{}.h5'.format(model_num, model_iter))

In [202]:
def gen_text(gen_length=400, diversity=1.0):
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    sys.stdout.write(sentence)
    for i in range(gen_length):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_int[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            
            preds = np.asarray(preds).astype('float64')
            preds = np.log(preds) / diversity
            exp_preds = np.exp(preds)
            preds = exp_preds / np.sum(exp_preds)
            probas = np.random.multinomial(1, preds, 1)
            next_index = np.argmax(probas)

            next_index = sample(preds, diversity)
            next_char = int_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
    print()
    

In [281]:
gen_text(gen_length=1000, diversity=0.5)

means that no one owns a United States come    The courts and so the bound of the common    The sent of my soul be the commorthand    That shall be a soldiers with his face    The service of a son and the company'd the crown    That shall be the court of the country    The more their bear with the courts be all the sense,    And then where we will be the bear thee    With the right of the sent of the country    The strikes of the common the subser with the bear with the bear thee seed    And when the world be a bold the court of the country    The server of the rest be son and so man,    The seem the courts be with the state    That the sent of his son of the stain's son    That is the great a would be so the sons.                                                                                                                                                                                                                                                                                    

In [262]:
import os

In [263]:

checkpoint_dir = os.path.dirname(checkpoint_path)

In [264]:
!pip install -q pyyaml h5py

In [266]:
model.save('model0.h5')

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN